In [1]:
# default_exp convert_to_zarr

# Notebook which converts per region netCDF files to Zarr files to make them more efficient when indexing

### uses pangeo_small environment

In [2]:
#test_ignore
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
#export
import xarray as xr
import zarr
from joblib import Parallel, delayed
import pandas as pd
import os
from numcodecs import Blosc

## Set the parameters here
### Ensure all State and Regions you want to transform are specified here. 

In [4]:
#test_ignore
data_root = '/media/scottcha/Data2/OAPMLData/'
output_root = '/media/scottcha/E1/Data/OAPMLData/'
interpolation = 1

#currently only have Washington regions and one season specified for the tutorial
#uncomment regions and seasons if doing a larger transform
regions = {#'Utah': ['Abajos', 'Logan', 'Moab', 'Ogden', 'Provo', 
           #'Salt Lake', 'Skyline', 'Uintas'],  
           'Colorado': ['Grand Mesa Zone', 'Sangre de Cristo Range', 'Steamboat Zone', 'Front Range Zone',
           'Vail Summit Zone', 'Sawatch Zone', 'Aspen Zone', 
           'North San Juan Mountains', 'South San Juan Mountains', 'Gunnison Zone'],
           'Washington': ['Mt Hood', 'Olympics', 'Snoqualmie Pass', 'Stevens Pass',
           #'WA Cascades East, Central', 'WA Cascades East, North', 'WA Cascades East, South',
           #'WA Cascades West, Central', 'WA Cascades West, Mt Baker', 'WA Cascades West, South'
           #]           
           'Canada': ["Northwest Coastal", "Northwest Inland", "Sea To Sky", 
                       "South Coast Inland", "South Coast", "North Rockies", 
                       "Cariboos", "North Columbia", "South Columbia", "Purcells", 
                       "Kootenay Boundary", "South Rockies", "Lizard Range and Flathead", 
                       "Vancouver Island", "Kananaskis Country, Alberta Parks", "Chic Chocs, Avalanche Quebec",
                       "Little Yoho", "Banff, Yoho and Kootenay National Parks", "Glacier National Park",
                       "Waterton Lakes National Park", "Jasper National Park"]
           }
seasons = ['15-16', '16-17', '17-18', '18-19', '19-20', '20-21']

In [5]:
#export
class ConvertToZarr:
    """
    Class which encapsulates the logic to convert a set of filtered netCDF files to Zarr
    """
    
    def __init__(self, seasons, regions, data_root, interpolate=1, resample_length='1d'):
        """
        Initialize the class
        
        Keyword Arguments
        seasons: list of season values to process
        regions: dictonary of Key: State and Value: List of Regions to process for that state
        data_root: the root path of the data folders which contains the 3.GFSFiltered1xInterpolation
        interpolate: the amount of interpolation applied in in the previous ParseGFS notebook (used for finding the correct input/output paths)
        """
        self.processed_path = data_root + '/3.GFSFiltered'+ str(interpolate) + 'xInterpolation' + resample_length + '/'
        self.zarr_base_path = output_root + '/4.GFSFiltered'+ str(interpolate) + 'xInterpolationZarr' + resample_length + '/'
        
        self.seasons = seasons
        self.regions = regions
        self.data_root = data_root
        self.compressor = Blosc(cname='zstd', clevel=3, shuffle=Blosc.BITSHUFFLE)
        
        if not os.path.exists(self.zarr_base_path):
            os.makedirs(self.zarr_base_path)
    
    def compute_region(self, region_name, season, state):
        """
        Calculates the zarr conversion for a specific region, season and state and indexes it for efficient lookup 
        
        Keyword Arguments
        region_name: name of the region to process
        season: season to process
        state: state to process (region must be a part of the state)
        """
        first = True
        base_path = self.processed_path + season + '/' + '/Region_' + region_name 
        zarr_path = self.zarr_base_path + season + '/' + state + '/Region_' + region_name + '.zarr'
        
        #TODO: refactor these to be shared code as logic also exists in ParseGFS
        p = 181
        if season in ['15-16', '19-20']:
            p = 182 #leap years
        
        if season == '15-16':
            snow_start_date = '2015-11-01'
        elif season == '16-17':
            snow_start_date = '2016-11-01'
        elif season == '17-18':
            snow_start_date = '2017-11-01'
        elif season == '18-19':
            snow_start_date = '2018-11-01'
        elif season == '19-20':
            snow_start_date = '2019-11-01'
        elif season == '20-21':
            snow_start_date = '2020-11-01'
        else:
            raise Exception('No known season ' + season)

        date_values_pd = pd.date_range(snow_start_date, periods=p, freq="D")
        try:
            with xr.open_zarr(zarr_path) as z:
                if z.time.values[-1] == date_values_pd[-1]:
                    print(' already exists: ' + region_name + ' ' + season + ' ' + state)
                    z.close()
                    return
                else:
                    #already exists but incomplete
                    date_values_pd = [pd.Timestamp(v) for v in date_values_pd.values.astype('datetime64[ns]') if v not in z.time.values]
                    print(' some exist but have to complete ' + str(len(date_values_pd)))
                    first = False
        except ValueError as err:
            #ignore as it doesn't exist yet
            pass
        
        #sometimes vars get added, filter to only the list of vars in the first dataset for that region
        #TODO: handle the case where the first dataset has more vars than subsequent ones
        final_vars = None
        last_ds = xr.open_dataset(base_path + '_' + date_values_pd[-1].strftime('%Y%m%d') + '.nc')
        #TODO: instead of variable-1, try vars -1
        last_ds = last_ds.to_array(name='vars').chunk({'time':-1, 'latitude':-1, 'longitude':-1, 'variable':-1}).to_dataset()
        last_vars = list(last_ds.variable.values)
        
        for d in date_values_pd:

            path =  base_path + '_' + d.strftime('%Y%m%d') + '.nc'
            print('On ' + str(path.split('/')[-1]))

            try:
                ds = xr.open_dataset(path, chunks={'latitude':1, 'longitude':1})
            except OSError as err:
                print(' missing file: ' + path)
                continue

            ds = ds.to_array(name='vars').chunk({'time':-1, 'latitude':-1, 'longitude':-1, 'variable':-1}).to_dataset()


            if first:
                #find intersection of the first and last variables to try to ensure
                #that we are using only the intersection
                final_vars = [v for v in list(ds.variable.values) if v in last_vars]
                ds = ds.sel(variable=ds.variable.isin(final_vars))
                enc = {x: {"compressor": self.compressor} for x in ds}
                ds.sortby('variable').to_zarr(zarr_path, consolidated=True, encoding=enc)
                first=False
            else:
                assert(final_vars is not None)
                ds = ds.sel(variable=ds.variable.isin(final_vars))                
                ds.sortby('variable').to_zarr(zarr_path, consolidated=True, append_dim='time')


    def process_tuple(self, t): 
        """
        Entry method to call compute_region with a tuple
        Basically a helper for executing with joblib parallel
        
        Keyword Arguments
        t: the tuple containing the region, season and state
        """
        self.compute_region(t[0], t[1], t[2])
    
    def make_list(self):
        """
        Helper method to make the list of values to process
        """
        to_process = []
        for s in self.seasons:
            for state in self.regions.keys():           
                for r in self.regions[state]:
                    to_process.append((r,s,state))
        return to_process
    
    def convert_local(self, jobs=15):
        l = self.make_list()
    
        #one state & season takes about 6 hours with 15 cores on my machine
        Parallel(n_jobs=jobs, backend="multiprocessing")(map(delayed(self.process_tuple), l))

In [12]:
#test_ignore
ctz = ConvertToZarr(seasons, regions, data_root, resample_length='3H')

In [13]:
%%time
#test_ignore
#1 region two seasons
#1 disk
#20 cores, no compression 
#CPU times: user 902 ms, sys: 472 ms, total: 1.37 s
#Wall time: 23min 36s
#103MB
#compression same time but 52MB

# all regions two seasons
# 2 disks
# 12 cores
# Wall Time 132 mins
# 1.2GB

#all regions(CO and CA) all  seasons
#2 disks
#20 cores
#CPU times: user 1min 48s, sys: 46.9 s, total: 2min 35s
#Wall time: 1d 12h 1min 33s
# 15gb for all data compressed

ctz.convert_local(jobs = 20)

/tmp/ipykernel_15402/2406378655.py:63: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  with xr.open_zarr(zarr_path) as z:


On Region_Vail Summit Zone_20151101.nc
On Region_Vail Summit Zone_20151102.nc
On Region_Vail Summit Zone_20151103.nc
On Region_Vail Summit Zone_20151104.nc
On Region_Vail Summit Zone_20151105.nc
On Region_Vail Summit Zone_20151106.nc
On Region_Vail Summit Zone_20151107.nc
On Region_Vail Summit Zone_20151108.nc
On Region_Vail Summit Zone_20151109.nc
On Region_Vail Summit Zone_20151110.nc
On Region_Vail Summit Zone_20151111.nc
On Region_Vail Summit Zone_20151112.nc
On Region_Vail Summit Zone_20151113.nc
On Region_Vail Summit Zone_20151114.nc
On Region_Vail Summit Zone_20151115.nc
On Region_Vail Summit Zone_20151116.nc
On Region_Vail Summit Zone_20151117.nc
On Region_Vail Summit Zone_20151118.nc
On Region_Vail Summit Zone_20151119.nc
On Region_Vail Summit Zone_20151120.nc
On Region_Vail Summit Zone_20151121.nc
On Region_Vail Summit Zone_20151122.nc
On Region_Vail Summit Zone_20151123.nc
On Region_Vail Summit Zone_20151124.nc
On Region_Vail Summit Zone_20151125.nc
On Region_Vail Summit Zon